In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 49s 1us/step


In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(21, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
train_data_dir = 'train path'
validation_data_dir = 'validation path'
image_size = (299, 299)

datagen = image.ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical')

Found 7350 images belonging to 21 classes.
Found 2100 images belonging to 21 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

In [ ]:
evaluation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

y_true = evaluation_generator.classes
y_pred = model.predict_generator(evaluation_generator, steps=evaluation_generator.samples)

In [10]:
y_pred = np.argmax(y_pred, axis=1)

In [11]:
accuracy = np.mean(y_true == y_pred)
report = classification_report(y_true, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.9142857142857143
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       100
           1       1.00      0.98      0.99       100
           2       0.96      0.94      0.95       100
           3       0.95      0.99      0.97       100
           4       1.00      0.68      0.81       100
           5       1.00      1.00      1.00       100
           6       0.73      0.82      0.77       100
           7       0.99      0.99      0.99       100
           8       0.87      1.00      0.93       100
           9       0.92      0.95      0.94       100
          10       0.99      0.99      0.99       100
          11       0.98      0.89      0.93       100
          12       0.59      0.93      0.72       100
          13       0.95      0.83      0.89       100
          14       0.93      0.91      0.92       100
          15       0.97      0.99      0.98       100
          16       0.95     

In [15]:
model.save('InceptionV3.h5')

In [17]:
image_size = (299, 299)

In [25]:
class_names = ['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential',
               'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumreesidential', 'mobilehomepark', 'overpass', 'parkinglot',
               'river', 'runway','sparseresidential', 'storagetanks', 'tenniscourt']

In [29]:
test_image_path = 'test image'
test_image = image.load_img(test_image_path, target_size=image_size)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

In [30]:
prediction = model.predict(test_image)
predicted_class_index = np.argmax(prediction, axis=1)
predicted_class_name = class_names[predicted_class_index[0]]

print("Predicted class:", predicted_class_name)

Predicted class: denseresidential
